Основано на блокноте https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb

In [1]:
! pip install datasets transformers seqeval corus razdel -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.1 MB/s eta 0:

В этом блокноте мы дообучаем модель на задаче классификации отдельных слов, а именно, распознавание именованных сущностей (aka named entity recognition, aka NER). Мы возьмём датасет медицинских сущностей, но в целом пайплайн подходит для любой задачи на выделение сущностей в тексте. 

Для скорости мы возьмём маленький BERT для русского языка [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny2); если взять другую, более крупную BERT-подобную модель, качество NER может быть выше, но и время обучения и работы будет дольше 


This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [2]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 16

## Loading the dataset

Для обучения мы возьмём [Russian Drug Reaction Corpus](https://github.com/cimm-kzn/RuDReC): размеченный корпус русскоязычных отзывов на лекарства. 

Загрузим мы его библиотекой corus, потому что это удобно 

In [3]:
from datasets import load_dataset, load_metric

In [4]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2023-03-23 14:42:50--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-03-23 14:42:50--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.008s  

2023-03-23 14:42:51 (224 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [5]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Пример документа:

In [6]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ), RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ), RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ), RuDReCEntity(
         entity_id='*[3]',
         entity_text='на веках высы

Посмотрим, какие сущности есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type

In [7]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [8]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Напишем функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд. 

In [9]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [10]:
print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [11]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

Пример данных

In [12]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
3164,"[А, вот, в, четвертый, и, пятый, раз, ,, мне, он, не, капельки, ничего, не, облегчил, простуда, проходила, по, полной, программе, со, всеми, симптомами, и, по, неделе, ,, хотя, принимала, также, при, первых, симптомах, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DI, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2104,"[Поставила, я, эту, свечку, сыну, на, ночь, .]","[O, O, O, B-Drugform, O, O, O, O]"
318,"[А, после, трех, дней, молочница, прошла, .]","[O, O, O, O, B-DI, O, O]"


Соберём все виды меток в список. 

In [13]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Сложим наши данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [14]:
from datasets import Dataset, DatasetDict

In [15]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You can directly call this tokenizer on one sentence:

In [21]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [18]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [22]:
example = ner_train[5]
print(example["tokens"])

['Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.']


In [23]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [24]:
len(example["tags"]), len(tokenized_input["input_ids"])

(16, 18)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [25]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [26]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-200 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


Here we set the labels of all special tokens to -200 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -200 to the other subtokens from the same word. We propose the two strategies here, just change the flag `label_all_tokens`.

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [83]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -200 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-200)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -200, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -200)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [84]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 1041, 37038, 33265, 19106, 40305, 22018, 548, 22276, 320, 21538, 16, 47886, 548, 59614, 11137, 626, 56606, 700, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-200, 0, 0, 0, 0, 1, -200, 7, 7, 7, 7, 0, 0, 0, 0, -200, -200, 0, -200, 0, -200]]}

In [29]:
ner_data['train'][22:23]

{'tokens': [['На',
   'следующее',
   'утро',
   'появились',
   'высыпания',
   'на',
   'лице',
   'и',
   'руках',
   ',',
   'похожие',
   'на',
   'комариные',
   'укусы',
   '.']],
 'tags': [['O',
   'O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [85]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [31]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [86]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized 

In [47]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwi

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [87]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [88]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [35]:
metric = load_metric("seqeval")

<ipython-input-35-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [36]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'DI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Drugform': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -200

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [37]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -200 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [39]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.65079665184021,
 'eval_precision': 0.009203364670954972,
 'eval_recall': 0.09830866807610994,
 'eval_f1': 0.016831056013030494,
 'eval_accuracy': 0.06279086946886311,
 'eval_runtime': 4.1774,
 'eval_samples_per_second': 230.285,
 'eval_steps_per_second': 14.602}

В начале обучения заморозим все параметры в модели, кроме последнего слоя, и посмотрим, насколько хорошо она обучится.

In [40]:
for param in model.bert.parameters():
    param.requires_grad = False

In [41]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[ 0.0141, -0.0182, -0.0077,  ...,  0.0190, -0.0034, -0.0175],
        [ 0.0142,  0.0104,  0.0043,  ...,  0.0189,  0.0233, -0.0325],
        [ 0.0171, -0.0060, -0.0334,  ..., -0.0354,  0.0007, -0.0195],
        ...,
        [ 0.0167, -0.0124, -0.0239,  ..., -0.0092, -0.0160,  0.0332],
        [-0.0131,  0.0109, -0.0095,  ...,  0.0165, -0.0410,  0.0134],
        [-0.0061,  0.0222,  0.0301,  ..., -0.0066, -0.0234, -0.0290]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


We can now finetune our model by just calling the `train` method:

In [42]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [89]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.320661,0.483165,0.303383,0.372727,0.920440
2,No log,0.276634,0.496689,0.396406,0.440917,0.927015
3,0.322400,0.254580,0.517564,0.467230,0.491111,0.932186
4,0.322400,0.238726,0.515541,0.508457,0.511974,0.935362
5,0.230900,0.225937,0.541893,0.526427,0.534048,0.939056
6,0.230900,0.218304,0.531096,0.568710,0.549260,0.940755
7,0.199100,0.209678,0.562000,0.594080,0.577595,0.944818
8,0.199100,0.202820,0.581892,0.604651,0.593053,0.946665
9,0.175900,0.198846,0.592145,0.621564,0.606498,0.948216
10,0.175900,0.195132,0.597847,0.645877,0.620935,0.949250


TrainOutput(global_step=4820, training_loss=0.17810758614441172, metrics={'train_runtime': 132.0257, 'train_samples_per_second': 582.765, 'train_steps_per_second': 36.508, 'total_flos': 49610055931590.0, 'train_loss': 0.17810758614441172, 'epoch': 20.0})

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморозим их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [44]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [45]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [46]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.323592,0.448649,0.263214,0.331779,0.917559
2,No log,0.280495,0.477361,0.390063,0.429319,0.925907
3,0.321200,0.257803,0.480403,0.453488,0.466558,0.930339
4,0.321200,0.240416,0.512332,0.483087,0.497280,0.933959
5,0.235000,0.227350,0.526425,0.536998,0.531659,0.937800
6,0.235000,0.217039,0.533663,0.569767,0.551125,0.939499
7,0.196900,0.209922,0.552632,0.599366,0.575051,0.942971
8,0.196900,0.203240,0.583920,0.614165,0.598660,0.946000
9,0.174200,0.198222,0.580676,0.635307,0.606764,0.946443
10,0.174200,0.195090,0.581662,0.643763,0.611139,0.946886


TrainOutput(global_step=4820, training_loss=0.17712433921845622, metrics={'train_runtime': 151.6319, 'train_samples_per_second': 507.413, 'train_steps_per_second': 31.788, 'total_flos': 49491316601520.0, 'train_loss': 0.17712433921845622, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.18183383345603943,
 'eval_precision': 0.6167146974063401,
 'eval_recall': 0.678646934460888,
 'eval_f1': 0.6462003019627579,
 'eval_accuracy': 0.9519834527591047,
 'eval_runtime': 0.7541,
 'eval_samples_per_second': 1275.718,
 'eval_steps_per_second': 80.893,
 'epoch': 20.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [91]:
p = trainer.predict(tokenized_datasets["test"])


predictions, labels, inputs = p.predictions, p.label_ids, tokenized_datasets["test"]['input_ids']
predictions = np.argmax(p.predictions, axis=2)

# send only the first token of each word to the evaluation
true_predictions = []
true_labels = []
for prediction, label, tokens in zip(predictions, labels, inputs):
    true_predictions.append([])
    true_labels.append([])
    for (p, l, t) in zip(prediction, label, tokens):
        if l != -200 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
            true_predictions[-1].append(label_list[p])
            true_labels[-1].append(label_list[l])

results = metric.compute(predictions=true_predictions, references=true_labels)
results

RuntimeError: ignored

In [90]:
p = trainer.predict(tokenized_datasets["test"])

RuntimeError: ignored

In [60]:
tokenized_datasets["test"][0]

{'tokens': ['Время', 'использования', ':', 'несколько', 'лет'],
 'tags': ['O', 'O', 'O', 'O', 'O'],
 'input_ids': [2, 24121, 16394, 30, 3910, 2262, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 0, 0, 0, 0, 0, -100]}

In [93]:
??trainer.predict

In [63]:
import torch

In [69]:
from torch.utils.data import DataLoader

In [71]:
data_loader= DataLoader(tokenized_datasets["train"], collate_fn=data_collator)

In [82]:
p[0][0]

array([[   1.2704258,   -0.8685294,   -0.7386838, ...,   -1.0974944,
          -1.2664338,   -1.5116389],
       [   1.5513592,   -0.963511 ,   -1.3720624, ...,   -2.1183054,
          -1.9732343,   -2.3021514],
       [   1.7679849,   -1.5396737,   -1.5408854, ...,   -1.5344515,
          -1.8109465,   -2.5072377],
       ...,
       [-100.       , -100.       , -100.       , ..., -100.       ,
        -100.       , -100.       ],
       [-100.       , -100.       , -100.       , ..., -100.       ,
        -100.       , -100.       ],
       [-100.       , -100.       , -100.       , ..., -100.       ,
        -100.       , -100.       ]], dtype=float32)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,12067,9,41,5,15,16,0,8,34,0,0,0,0
B-ADR,37,61,46,0,0,0,0,3,2,0,0,0,0
B-DI,58,9,192,1,2,5,0,0,3,0,0,0,0
B-Drugclass,5,1,11,53,2,1,0,0,0,0,0,0,0
B-Drugform,10,0,0,0,160,2,0,0,0,0,0,0,0
B-Drugname,4,0,2,0,1,219,0,0,0,0,0,0,0
B-Finding,10,15,29,0,0,1,0,0,1,0,0,0,0
I-ADR,57,6,3,0,0,0,0,59,44,0,0,0,0
I-DI,71,3,20,0,0,0,0,8,74,0,0,0,0
I-Drugclass,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Применение модели

In [77]:
import torch

In [78]:

text = ' '.join(ner_test[4]['tokens'])  # 458 is example of a difficult text
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [79]:
import torch

In [96]:
tokens = tokenizer(text, return_tensors='pt', padding="max_length").to(model.device)

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 2048, 13])

In [99]:
tokens

{'input_ids': tensor([[    2, 40210,  9292,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}

In [100]:
pred.logits[0][-2]

tensor([-0.0376, -0.0460,  0.1516,  0.8221, -0.2297,  0.1721,  0.0754, -0.0165,
        -0.4122, -0.7196, -0.3968,  0.2191,  0.5515])

In [81]:
pred.logits

tensor([[[ 1.1997, -0.9527, -0.4431, -1.0281, -0.9207, -1.0110, -1.0999,
          -1.2724, -1.1781, -0.8699, -1.0991, -1.6285, -1.3255],
         [ 0.5434, -1.2212, -0.2893, -1.6609, -0.0141, -0.4438, -1.3171,
          -0.8089, -1.2121, -1.4036, -1.7505, -1.4758, -1.3081],
         [ 1.7376, -2.0434, -0.6365, -2.1171, -1.2925, -1.6624, -1.7484,
          -1.6414, -1.9432, -1.7880, -1.9012, -1.6704, -2.4759],
         [ 1.6497, -1.8110, -0.8329, -2.1349, -1.2408, -1.5319, -2.4585,
          -1.5278, -1.9129, -1.6367, -1.7163, -1.8403, -2.3354],
         [ 1.6029, -1.6224, -1.0158, -1.8663, -1.5376, -0.9885, -1.8728,
          -1.5728, -1.5486, -0.8804, -1.8134, -2.0439, -1.5611],
         [ 2.0476, -2.1573, -1.0785, -2.4259, -1.2516, -1.3429, -2.0751,
          -1.6022, -1.8261, -1.8464, -2.1231, -2.5671, -2.0063],
         [ 2.0223, -1.6933, -1.0633, -1.8402, -1.2458, -1.3886, -1.9200,
          -1.5550, -1.1131, -1.5385, -2.0110, -1.9111, -1.7843],
         [ 1.9184, -2.0149, -0.773

In [ ]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
Охо             O         
##тно           O         
применя         O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
насмор          B-DI      
##ком           I-DI      
,               O         
что             O         
в               O         
моем            O         
случае          O         
явление         O         
очень           O         
частое          O         
.               O         
[SEP]           O         


Более простое применение модели: пайплайн от huggingface

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [ ]:
print(text)
print(pipe(text))

Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .
[{'entity_group': 'DI', 'score': 0.4562955, 'word': 'насморком', 'start': 33, 'end': 42}]


# Самостоятельная работа

Попробуйте поднять F1 на тестовой выборке. 

Что можно пробовать:
* Изменить параметры обучения (число эпох, learning rate, и т.п.)
* Подкладывать при инициализации другие модели (в т.ч. более крупные)
* По-другому размечать subword tokens (например, выставить label_all_tokens=True)
* Аугментировать данные